<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=300px/>
<br />
<h1>Оптимизация по времени и памяти</h1>
<br />
<h4>2023</h4>
</center>

# Почему Python не очень быстрый

Python - очень гибкий язык. Однако именно эта гибкость не позволяет делать многие оптимизации. <br>
Эффективные оптимизации закладываются на предположения и ограничения. <br>
Меньше ограничений - меньше простора для оптимизации.

## 1. Динамическая типизация

Чему это мешает:
* Много проверяем в Runtime. Тратим время.
* Не знаем точно с чем работаем - должны все время честно исполнять весь код

## 2. Изменяемость всего и вся

Несколько примеров:

In [1]:
import builtins

print(len("abc"))
len = lambda obj: "mock!"
print(len("abc"))
len = builtins.len

3
mock!


In [2]:
def my_func(a, b):
    return a + b

print(my_func(1, 2))

def new_func(a, b):
    return a * b

my_func.__code__ = new_func.__code__
print(my_func(1, 2))

3
2


In [3]:
import sys
import ctypes

def change_local_variable():
    # Get prev frame object from the caller
    frame = sys._getframe(1)
    frame.f_locals['my_var'] = "hello"
    # Force update
    ctypes.pythonapi.PyFrame_LocalsToFast(ctypes.py_object(frame),
                                          ctypes.c_int(0))

def do_smth():
    my_var = 1
    change_local_variable()
    print(my_var)

    
do_smth()

hello


**Следствие: честно исполняем код**

In [4]:
def do1():
    a = [-1] * 1000
    for i in range(len(a)):
        if i == 0:
            a[i] = 1
        else:
            a[i] = i
            
def do2():
    a = [-1] * 1000
    a[0] = 1
    for i in range(1, len(a)):
        a[i] = i

In [5]:
%timeit -n100 do1()
%timeit -n100 do2()

18.6 µs ± 516 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)
13.2 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 3. CPython

1. Старый проект, написан задолго до многоядерных процессоров и т.д.
2. Производительность - не самая главная цель
3. Необходимость поддерживать совместимость с C API (особенности внутреннего дизайна)

Но есть и хорошое:

https://docs.python.org/3/whatsnew/3.11.html#summary-release-highlights

https://docs.python.org/3/whatsnew/3.11.html#faster-cpython
   

Будущее:
1. https://github.com/faster-cpython/
1. Multithreaded Python without the GIL - https://docs.google.com/document/d/18CXhDb1ygxg-YXNBJNzfzZsDFosB5e6BfnXLlejd9l0/edit#

# Когда оптимизировать

### *Premature optimization is the root of all evil*
Так ли это?

**Утверждение:**

В первую очередь мы пишим работающий код, а потом быстрый. Мы будем заниматься оптимизацией, когда функционал будет готов.

**Следствие:**

Производительность будет одна и та же все время, пока кто-то не найдет легко исправимые вещи, которые сделаю программу быстрее без неоходимости переделки большого количества кода.

Может повезти, а может не повезти.

**Правильный путь:**

Если вам нужна быстрая программа - сразу обращайте внимание на производительность. <br>
Ваш прототип должен быть быстрый - может даже быстрее, чем финальная версия.

Лучше начать с производительного решения и поддерживать его, чем надеятся, что получится оптимизировать медленное решение.

**Антипаттерн: большой комок грязи**
```
If you think good architecture is expensive, try bad architecture.
```


http://www.laputan.org/mud/mud.html
https://ru.wikipedia.org/wiki/%D0%91%D0%BE%D0%BB%D1%8C%D1%88%D0%BE%D0%B9_%D0%BA%D0%BE%D0%BC%D0%BE%D0%BA_%D0%B3%D1%80%D1%8F%D0%B7%D0%B8

## Мантра оптимизаций

1. Не делай
2. Делай это позже
3. Делай это оптимально

# Как оптимизировать

<center><img src="http://lh6.ggpht.com/_AALI9OaE6pk/Sjio4NqVK0I/AAAAAAAAAEM/9xwU-xHtEBY/s800/premature2.PNG">
<a href="https://dl.acm.org/citation.cfm?doid=356635.356640"> Knuth, D. E. 1974. Structured Programming with go to Statements</a>, ACM Comput. Surv. 6, 4 (Dec. 1974), 261-301.</center>


**Нужно найти место, куда прикладывать усилия!**

## Правило 1. Профилируй код

Возможно вы оптимизируете какую-то функцию в 10 раз. <br>
Однако она исполняется всего в 1% случаев.  <br>
В итоге польза от такой оптимизации довольно маленькая.

Не надо гадать какая часть чаще всего используется и дольше всего работает. <br>
Профилирование позволяет понять какая именно часть нужно оптимизировать.


## Правило 2. Не забывай про корректность

Ваши оптимизации вполне могут сломать код. <br>
Стоит покрыть дополнительными тестами те части, которые вы хотите поменять.

# Профилирование

Основной инструментарий:
1. cProfile
2. pstats
3. SnakeViz

Profile demo

Дополнительно хочется выделить два инструмента:
1. <a href="https://github.com/benfred/py-spy">py-spy</a> - позволяет снять профиль с работающей программы, без изменений кода
2. <a href="https://github.com/pyutils/line_profiler">line_profiler</a> - профилирование по строчкам (показывает количество времени проведенную в каждой строчке)

# Измерение времени

Иногда хочется просто замерить время, а не снимать полноценный профиль. <br>
Например, когда мы оптимизируем одну конкретную функцию.
Для этого есть модуль `timeit`

In [6]:
import timeit

setup = '''
s='abcdefghijklmnopqrstuvwxyz'

def reverse_0(s: str) -> str:
    reversed_output = ''
    s_length = len(s)
    for i in range(s_length-1, 0-1, -1):
        reversed_output = reversed_output + s[i]
    return reversed_output

def reverse_5(s: str) -> str:
    return s[::-1]
'''

In [7]:
timeit.timeit('reverse_0(s)', setup, number=10000)

0.008076874999460415

In [8]:
timeit.timeit('reverse_5(s)', setup, number=10000)

0.0005927920010435628

Функция `timeit` замеряет время с помощью функции `time.perf_counter`. <br>
На время измерений отключается сборщик мусора. <br>
При этом замеряется общее время нужное для `N` запусков, а не среднее.

Q: Почему все в строках?

A: Сам код `timeit` сделан в виде <a href="https://github.com/python/cpython/blob/master/Lib/timeit.py#L69">шаблоннонй строки</a>, куда подставляются параметры. <br>
Это позволяет сэкономить время на вызове функции, если бы мы ее передавали в виде объекта. <br>
В `timeit` можно передавать и функции по честному.

В IPython есть упрощение работы с функцией `timeit` - специальная команда `%timeit`. <br>
В отличии от функции эта команда выводит среднее время работы и стандартное отклонение.

In [9]:
def reverse_0(s: str) -> str:
    reversed_output = ''
    s_length = len(s)
    for i in range(s_length-1, 0-1, -1):
        reversed_output = reversed_output + s[i]
    return reversed_output

%timeit -n100 reverse_0('abcdefghijklmnopqrstuvwxyz')

793 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Оптимизация

## Часть 1. Что оптимизировать

Оптимизация - это не только изменение кода. <br>
Можно выделить следующие уровни оптимизации:

### 1. Общая архитектура

То как система работает. Какие данные обрабатываются, как обрабатываются, объем данных, хранение и т.д.

### 2. Алгоритмы и структуры данных

Выбор того или иного алгоритма/структуры данных при обработке.

### 3. Реализация (код)

Непосредственная реализация алгоритма/структуры данных

### 4. Оптимизации во время компиляции

### 5. Оптимизации во время исполнения 

Мы будем обсуждать оптимизации на уровнях 3-5. <br>
Однако оптимизации на уровне 1-2 тоже важны. <br>
Более того у них больший потенциал для ускорения, но в тоже время они наиболее сложные.

В целом оптимизация - это не только про скорость, но еще и:
* Память
* Диск (место, I\O)
* Сеть
* Потребление энергии
* И многое другое

Мы обсудим только скорость работы и память.


Оптимизация - может быть сложной.

1. На оптимизацию тратится время. Кроме того не факт что ваши оптимизации что-то то дадут
1. Скорее всего система в целом станет сложнее, а код непонятнее
1. Не любые оптимизации полезны: можно выиграть скорость, но существенно проиграть память

## Часть 2. Пишем хороший Python код

Будем оптимизировать 3 уровень - реализацию (код).

### Совет 1. Используй builtins

Посчитаем количество элементов в списке:

In [10]:
one_million_elements = [i for i in range(1000000)]

def calc_total(elements):
    total = 0
    for item in elements:
        total += 1
    
%timeit calc_total(one_million_elements)

14.6 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
%timeit len(one_million_elements)

15.9 ns ± 0.034 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


Пример выше - игрушечный. <br>
Однако в большинстве случаев вместо того, чтобы писать что-то свое лучше использовать готовую функцию из `builtins.`

### Совет 2. Правильная фильтрация

Попробум получить новый список, отфильтровав только нечетные элементы. <br>
Кроме того воспользуемся предыдущим советом и будем использовать `filter` из `builtins`.

In [12]:
def my_filter1(elements):
    result = []
    for item in elements:
        if item % 2:
            result.append(item)
    return result
            
def my_filter2(elements):
    return list(filter(lambda x: x % 2, elements))

In [13]:
%timeit my_filter1(one_million_elements)

17.6 ms ± 8.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%timeit my_filter2(one_million_elements)

28 ms ± 60 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Q: Почему код стал медленнее?

A: Потому что у нас есть накладные расходы на создание генератора, а потом превращения генератора в список.

Давайте напишим код, который лучше отражает наши намеренья и будет сразу создавать нужный список:

In [15]:
def my_filter3(elements):
    return [item for item in elements if item % 2]

%timeit my_filter3(one_million_elements)

15.9 ms ± 22.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
one_million_elements_str = [str(i) for i in range(1000000)]

def str_filter1(elements):
    return [item for item in elements if item.isdigit()]

def str_filter2(elements):
    return list(filter(str.isdigit, elements))

In [17]:
%timeit str_filter1(one_million_elements_str)

24.7 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%timeit str_filter2(one_million_elements_str)

19.7 ms ± 38.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Мораль: Не всегда использование `builtins` и генераторов делает код быстрее. <br>
Стоит проверять конкретно ваш случай.

### Совет 3. Правильная проверка вхождений

Напишим код, проверяющий наличие элемента:

In [19]:
def check_in1(elements, number):
    for item in elements:
        if item == number:
            return True
    return False

%timeit check_in1(one_million_elements, 500000)

3.95 ms ± 1.89 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%timeit 500000 in one_million_elements

2.51 ms ± 8.38 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Однако, время поиска зависит от того, где именно находится элемент

In [21]:
%timeit 42 in one_million_elements

223 ns ± 1.92 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


В Python есть `set` - стандартный инструмент для такой задачи

In [22]:
one_million_elements_set = set(one_million_elements)
%timeit 500000 in one_million_elements_set

16.2 ns ± 0.00751 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [23]:
%timeit 42 in one_million_elements_set

11.4 ns ± 0.00171 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


Однако, конечно же, мы проиграем время при создании множества:

In [24]:
%timeit set(one_million_elements)

10.1 ms ± 47.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Кроме того, конечно же мы проиграли память.

### Совет 4. Сортировка

In [25]:
%timeit sorted(one_million_elements)

4.96 ms ± 26.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
%timeit one_million_elements.sort()

2.72 ms ± 3.82 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Мораль: inplace сортировка заметно быстрее. При возможности пользуйтесь именно ей.

### Совет 5. Условия if

Условия в конструкции if можно писать по разному:

In [27]:
count = 100000

def check_false1(flag):
    for i in range(count):
        if flag == False:
            pass
    
def check_false2(flag):
    for i in range(count):
        if flag is False:
            pass

def check_false3(flag):
    for i in range(count):
        if not flag:
            pass

При этом эти варианты работают разное время:

In [28]:
%timeit check_false1(True)

1.53 ms ± 3.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
%timeit check_false2(True)

1.25 ms ± 4.41 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [30]:
%timeit check_false3(True)

1.02 ms ± 7.01 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Попробуем угадать какой способ проверки на пустоту быстрее:
1. `if len(elements) == 0:`
2. `if elements == []:`
3. `if not element:`

In [31]:
def check_empty1(elements):
    for i in range(count):
        if len(elements) == 0:
            pass
    
def check_empty2(elements):
    for i in range(count):
        if elements == []:
            pass

def check_empty2_new(elements):
    for i in range(count):
        if elements == list():
            pass
        
def check_empty3(elements):
    for i in range(count):
        if not elements:
            pass

In [32]:
%timeit check_empty1(one_million_elements)

2.21 ms ± 610 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [33]:
%timeit check_empty2(one_million_elements)

1.99 ms ± 660 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
%timeit check_empty2_new(one_million_elements)

3.23 ms ± 59.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
%timeit check_empty3(one_million_elements)

1.12 ms ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Мораль: пользуйтесь самым быстрым способом. Кроме производительности этот способ более Python-way.

### Совет 6. Спрашивать разрешения или обрабатывать последствия

Предпололжим мы хотим написать код, который будет обрабатывать объекты как имеющие некоторый аттрибут, так и нет.

In [36]:
class Foo:
    attr1 = 'hello'
    
foo = Foo()

In [37]:
def check_attr1(obj):
    for i in range(count):
        if hasattr(obj, 'attr1'):
            obj.attr1
            
def check_attr2(obj):
    for i in range(count):
        try:
            obj.attr1
        except AttributeError:
            pass

Какой способ быстрее?

In [38]:
%timeit check_attr1(foo)

3.33 ms ± 22.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [39]:
%timeit check_attr2(foo)

1.9 ms ± 6.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Разница станет еще большей, если нужно будет проверять несколько аттрибутов.

Где подвох?

Предположим, что у объектов в основном нет нужного аттрибута.

In [40]:
class Bar:
    pass

bar = Bar()

In [41]:
%timeit check_attr1(bar)

2.4 ms ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
%timeit check_attr2(bar)

23.5 ms ± 36 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Мораль: думайте какая ситуация чаще встречается и исходя из этого выбирайте из двух вариантов. <br>

Этот принцип работает для всех ситуаций, например, при создании запроса по сети.

### Совет 7. Особенности определения словаря и списка

В Python можно по разному объявлять словарь и список:

In [43]:
def create_list1():
    for i in range(count):
        a = []

def create_list2():
    for i in range(count):
        a = list()
        
def create_dict1():
    for i in range(count):
        a = {}

def create_dict2():
    for i in range(count):
        a = dict()

При этом способы через `[]` и `{}` быстрее `list()` и `dict()` соответственно: 

In [44]:
%timeit create_list1()

1.48 ms ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [45]:
%timeit create_list2()

2.68 ms ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [46]:
%timeit create_dict1()

1.55 ms ± 747 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [47]:
%timeit create_dict2()

2.94 ms ± 40.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Q: Почему есть разница?

A: Обращение к имени занимает время. Интерпретатору нужно найти на что указывает имя. Можно посмотреть на код через модуль `dis` и убедиться, что код разный.

### Совет 8. Вызов функции

Если есть возможность не вызывать функцию - лучше это сделать. <br>
Вызов функции и создание frame требует значительного количества времени.

In [48]:
def square(num):
    return num ** 2

In [49]:
%timeit [square(num) for num in range(10000)]

454 µs ± 2.38 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [50]:
%timeit [num ** 2 for num in range(10000)]

298 µs ± 290 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Совет 9. Избегайте активной работы с глобальными переменными

In [51]:
count = 100000

some_global = 0
def work_with_global():
    global some_global
    for i in range(count):
        some_global += 1
        
def work_with_local():
    some_local = 0
    for i in range(count):
        some_local += 1

In [52]:
%timeit work_with_global()

2.85 ms ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
%timeit work_with_local()

1.92 ms ± 1.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [54]:
some_global = 0
def work_with_global_optimized():
    global some_global
    some_local = some_global
    for i in range(count):
        some_local += 1
    some_global = some_local

In [55]:
%timeit work_with_global_optimized()

1.91 ms ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Совет 10. Для математики используйте соответвующие библиотеки

Не надо пытаться писать математические вычисления на Python. <br>
Используйте готовые библиотеки, которые написаны на C\Fortran

In [56]:
def list_slow():
    a = range(10000)
    return [i ** 2 for i in a]

%timeit list_slow()

298 µs ± 466 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [57]:
import numpy as np

def list_fast():
    a = np.arange(10000)
    return a ** 2

%timeit list_fast()

17.6 µs ± 675 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### <font color=red>Danger zone warning</font>
Используйте советы ниже только если это действительно даст какой-то сущетсвенный выигрыш

### Совет 11. Множественное присваивание

In [58]:
def create_variables1():
    for i in range(10000):
        a = 0
        b = 1
        c = 2
        d = 3
        e = 4
        f = 5
        g = 6
        h = 7
        i = 8
        j = 9
        
def create_variables2():
    for i in range(10000):
        a, b, c, d, e, f, g, h, i, j = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

In [59]:
%timeit create_variables1()

270 µs ± 194 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [60]:
%timeit create_variables2()

232 µs ± 4.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Объявление переменных на одной строчки работает действительно быстрее, но не стоит так делать

### Совет 11. Поиск функций и аттрибутов

В Python поиск аттрибута сложная операция. Вызывается `__getattr__` и `__getattribute__`. <br>
Можно найти аттрибут один раз и сохранить его, чтобы не искать повторно:

In [61]:
def squares1(elements):
    result = []
    for item in elements:
        result.append(item)

def squares2(elements):
    result = []
    append = result.append
    for item in elements:
        append(item)

In [62]:
%timeit squares1(one_million_elements)

11.1 ms ± 26.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
%timeit squares2(one_million_elements)

12.9 ms ± 126 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Часть 2.5 Пробуем профилировать вычисления

Попробуем оптимизировать код, который занимается перемножением матриц.

In [64]:
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return (len(self), len(self[0]))

In [65]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    
    assert n_xcols == n_yrows, "Incompatible matrix dimensions"
    
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z

In [66]:
X = Matrix([[1], [2], [3]])
Y = Matrix([[4, 5, 6]])
print(matrix_product(X, Y))
print(matrix_product(Y, X))

[[4, 5, 6], [8, 10, 12], [12, 15, 18]]
[[32]]


In [67]:
shape = (100, 100)

X = Matrix.random(shape)
Y = Matrix.random(shape)

In [68]:
%timeit matrix_product(X, Y)

81.7 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Текущий код - медленный. <br>
Попробуем попрофилировать и найти причину.

In [69]:
import cProfile
source = open("benchmark.py").read()
cProfile.run(source, sort="tottime")

         161351 function calls in 15.078 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100   15.051    0.151   15.052    0.151 <string>:26(matrix_product)
    20000    0.008    0.000    0.016    0.000 random.py:284(randrange)
    20000    0.005    0.000    0.006    0.000 random.py:235(_randbelow_with_getrandbits)
        1    0.004    0.004   15.078   15.078 <string>:41(main)
    20000    0.003    0.000    0.019    0.000 random.py:358(randint)
      200    0.002    0.000    0.022    0.000 <string>:12(<listcomp>)
    60000    0.002    0.000    0.002    0.000 {built-in method _operator.index}
      100    0.001    0.000    0.001    0.000 <string>:6(<listcomp>)
    20043    0.001    0.000    0.001    0.000 {method 'getrandbits' of '_random.Random' objects}
    20000    0.001    0.000    0.001    0.000 {method 'bit_length' of 'int' objects}
        1    0.000    0.000   15.078   15.078 {built-in method builtins.exec}
  

Понятнее не стало. Воспользуемся более удобным инструментом - line_profiler.

In [70]:
%load_ext line_profiler
def bench(shape=(100, 100), n_iter=10):
    X = Matrix.random(shape)
    Y = Matrix.random(shape)
    for iter in range(n_iter):
        matrix_product(X, Y)

In [71]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        for j in range(n_xcols):
            for k in range(n_ycols):
                Z[i][k] += X[i][j] * Y[j][k]
    return Z  

%lprun -f matrix_product bench()

Timer unit: 1e-09 s

Total time: 3.38736 s
File: /var/folders/_2/xfr6q9x92bv26fgxk9nk2g9w0000gq/T/ipykernel_8393/2668735827.py
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def matrix_product(X, Y):
     2        10      29000.0   2900.0      0.0      n_xrows, n_xcols = X.shape
     3        10       2000.0    200.0      0.0      n_yrows, n_ycols = Y.shape
     4        10     228000.0  22800.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5      1010      88000.0     87.1      0.0      for i in range(n_xrows):
     6    101000   10787000.0    106.8      0.3          for j in range(n_xcols):
     7  10100000  796570000.0     78.9     23.5              for k in range(n_ycols):
     8  10000000 2579653000.0    258.0     76.2                  Z[i][k] += X[i][j] * Y[j][k]
     9        10       1000.0    100.0      0.0      return Z

```
Timer unit: 1e-06 s

Total time: 19.6588 s
File: <ipython-input-71-a7f1e072bc31>
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product(X, Y):
     2        10        202.0     20.2      0.0      n_xrows, n_xcols = X.shape
     3        10         43.0      4.3      0.0      n_yrows, n_ycols = Y.shape
     4        10       1784.0    178.4      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5      1010        641.0      0.6      0.0      for i in range(n_xrows):
     6    101000      70509.0      0.7      0.4          for j in range(n_xcols):
     7  10100000    6018210.0      0.6     30.6              for k in range(n_ycols):
     8  10000000   13567378.0      1.4     69.0                  Z[i][k] += X[i][j] * Y[j][k]
     9        10         10.0      1.0      0.0      return Z
```

Видно, что большая часть времени проходит внутри самого внутреннего цикла. <br>
Попробуем его оптимизировать. <br>
Для начала - избавимся от обращения к индексу каждый раз (вызов `__getitem__` не бесплатный).

In [72]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        Zi = Z[i]
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += Xi[j] * Y[j][k]
            Zi[k] = acc
    return Z

In [73]:
%timeit matrix_product(X, Y)

44.8 ms ± 33.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Стало лучше. Попрофилируем еще раз.

In [74]:
%lprun -f matrix_product bench()

Timer unit: 1e-09 s

Total time: 2.53289 s
File: /var/folders/_2/xfr6q9x92bv26fgxk9nk2g9w0000gq/T/ipykernel_8393/1572409396.py
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def matrix_product(X, Y):
     2        10       8000.0    800.0      0.0      n_xrows, n_xcols = X.shape
     3        10       8000.0    800.0      0.0      n_yrows, n_ycols = Y.shape
     4        10     190000.0  19000.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5      1010      84000.0     83.2      0.0      for i in range(n_xrows):
     6      1000     147000.0    147.0      0.0          Xi = X[i]
     7      1000     143000.0    143.0      0.0          Zi = Z[i]
     8    101000   10057000.0     99.6      0.4          for k in range(n_ycols):
     9    100000    7685000.0     76.8      0.3              acc = 0
    10  10100000  838670000.0     83.0     33.1              for j in range(n_xcols):


```
Timer unit: 1e-06 s

Total time: 18.3915 s
File: <ipython-input-72-f560ca41ce9b>
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product(X, Y):
     2        10         75.0      7.5      0.0      n_xrows, n_xcols = X.shape
     3        10         25.0      2.5      0.0      n_yrows, n_ycols = Y.shape
     4        10        726.0     72.6      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5      1010       1254.0      1.2      0.0      for i in range(n_xrows):
     6      1000        929.0      0.9      0.0          Xi = X[i]
     7      1000       1557.0      1.6      0.0          Zi = Z[i]
     8    101000      69899.0      0.7      0.4          for k in range(n_ycols):
     9    100000      69961.0      0.7      0.4              acc = 0
    10  10100000    6506929.0      0.6     35.4              for j in range(n_xcols):
    11  10000000   11658221.0      1.2     63.4                  acc += Xi[j] * Y[j][k]
    12    100000      81917.0      0.8      0.4              Zi[k] = acc
    13        10          5.0      0.5      0.0      return Z
 ```

Практически все время проходит в самом внутреннем списке. <br>
Попробуем сделать быстрее.

In [75]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    for i in range(n_xrows):
        Xi = X[i]
        Zi = Z[i]
        for k in range(n_ycols):
            Zi[k] = sum(Xi[j] * Y[j][k] for j in range(n_xcols))
    return Z

In [76]:
%timeit matrix_product(X, Y)

56 ms ± 89.5 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [77]:
%lprun -f matrix_product bench()

Timer unit: 1e-09 s

Total time: 1.66556 s
File: /var/folders/_2/xfr6q9x92bv26fgxk9nk2g9w0000gq/T/ipykernel_8393/2945563094.py
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def matrix_product(X, Y):
     2        10       7000.0    700.0      0.0      n_xrows, n_xcols = X.shape
     3        10       3000.0    300.0      0.0      n_yrows, n_ycols = Y.shape
     4        10     180000.0  18000.0      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5      1010      86000.0     85.1      0.0      for i in range(n_xrows):
     6      1000     120000.0    120.0      0.0          Xi = X[i]
     7      1000     111000.0    111.0      0.0          Zi = Z[i]
     8    101000    9324000.0     92.3      0.6          for k in range(n_ycols):
     9    100000 1655724000.0  16557.2     99.4              Zi[k] = sum(Xi[j] * Y[j][k] for j in range(n_xcols))
    10        10          0.0      0.0   

```
Timer unit: 1e-06 s

Total time: 6.63748 s
File: <ipython-input-75-2ec304f5e9af>
Function: matrix_product at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           def matrix_product(X, Y):
     2        10         68.0      6.8      0.0      n_xrows, n_xcols = X.shape
     3        10         28.0      2.8      0.0      n_yrows, n_ycols = Y.shape
     4        10        729.0     72.9      0.0      Z = Matrix.zeros((n_xrows, n_ycols))
     5      1010        672.0      0.7      0.0      for i in range(n_xrows):
     6      1000        954.0      1.0      0.0          Xi = X[i]
     7      1000       1025.0      1.0      0.0          Zi = Z[i]
     8    101000      80761.0      0.8      1.2          for k in range(n_ycols):
     9    100000    6553241.0     65.5     98.7              Zi[k] = sum(Xi[j] * Y[j][k] for j in range(n_xcols))
    10        10          6.0      0.6      0.0      return Z
```

Уберем из него еще одно обращение по индексу, транспонировав матрицу Y. <br>

In [78]:
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # better cell extraction
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

In [79]:
%timeit matrix_product(X, Y)

41.9 ms ± 29.2 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Конечно пример выше тоже не настоящий. <br>
В правильном коде надо делать так: `X.dot(Y)`

In [80]:
shape = (100, 100)
X2 = np.random.randint(-255, 255, size=shape)
Y2 = np.random.randint(-255, 255, size=shape)
%timeit X2.dot(Y2)

334 µs ± 693 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Часть 3. Оптимизации почти без изменений кода

<img src="http://numba.pydata.org/_static/numba-blue-horizontal-rgb.svg" width=500px/>

Numba - делает jit-компиляцию. В момент первого вызова код транслируется в LLVM и компилируется в машинный код. <br>
Декоратор `numba.jit` пытается вывести типы аргументов и возвращаемого значения декорируемой функции

К сожалению Numba плохо работает со списками. <br>
Q: Почему?

A: В Python списки гетерогенные. Нельзя гарантировать один и тот же тип у всех элементов. <br>
Поэтому перепишим наш код на `numpy.ndarray`

In [81]:
import numpy as np

def non_jit_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += X[i, j] * Y[j, k]
            Z[i, k] = acc
    return Z

In [82]:
shape = 100, 100
X3 = np.random.randint(-255, 255, shape)
Y3 = np.random.randint(-255, 255, shape)

In [83]:
%timeit non_jit_matrix_product(X3, Y3)

172 ms ± 126 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [84]:
import numba

@numba.jit(cache=True, nopython=True)
def jit_matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    for i in range(n_xrows):
        for k in range(n_ycols):
            acc = 0
            for j in range(n_xcols):
                acc += X[i, j] * Y[j, k]
            Z[i, k] = acc
    return Z

In [85]:
%timeit jit_matrix_product(X3, Y3)

338 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Скомпилированная версия стала заметно быстрее.

Можно посмотреть, какие типы вывела Numba:

In [86]:
jit_matrix_product.inspect_types()

jit_matrix_product (Array(int64, 2, 'C', False, aligned=True), Array(int64, 2, 'C', False, aligned=True))
--------------------------------------------------------------------------------
# File: /var/folders/_2/xfr6q9x92bv26fgxk9nk2g9w0000gq/T/ipykernel_7993/4100080029.py
# --- LINE 3 --- 
# label 0
#   X = arg(0, name=X)  :: array(int64, 2d, C)
#   Y = arg(1, name=Y)  :: array(int64, 2d, C)

@numba.jit(cache=True, nopython=True)

# --- LINE 4 --- 

def jit_matrix_product(X, Y):

    # --- LINE 5 --- 
    #   $6load_attr.1 = getattr(value=X, attr=shape)  :: UniTuple(int64 x 2)
    #   $16unpack_sequence.4 = exhaust_iter(value=$6load_attr.1, count=2)  :: UniTuple(int64 x 2)
    #   del $6load_attr.1
    #   $16unpack_sequence.2 = static_getitem(value=$16unpack_sequence.4, index=0, index_var=None, fn=<built-in function getitem>)  :: int64
    #   $16unpack_sequence.3 = static_getitem(value=$16unpack_sequence.4, index=1, index_var=None, fn=<built-in function getitem>)  :: int64
    #   de

Numba хорошо подходит для различных вычислений. Однако, при работе например со строками все скорее замедлится:

In [87]:
def create_request_url(addr, rest_method, use_https):
    for i in range(10000):
        url = 'https://' if use_https else 'http://'
        url += addr
        if rest_method is not None:
            url += '/?' + rest_method
    return url

In [88]:
%timeit create_request_url('127.0.0.1', 'ping', True)

709 µs ± 8.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [89]:
@numba.jit(cache=True, nopython=True)
def jit_create_request_url(addr, rest_method, use_https):
    for i in range(10000):
        url = 'https://' if use_https else 'http://'
        url += addr
        if rest_method is not None:
            url += '/?' + rest_method
    return url

In [90]:
%timeit jit_create_request_url('127.0.0.1', 'ping', True)

907 µs ± 3.35 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Это не все возможности Numba. <br>
Что мы не затронули:
* AOT компиляция
* Освобождение GIL
* Аннотация типов
* Работа с классами
* и т.д.

Подробнее на странице проекта: http://numba.pydata.org.

### Часть 4. Cython

Cython это:
* Расширение языка Python с типизацией. При этом любой код на Python - корректный код на Cython
* Оптмизирующий компилятор Python и Cython в код на C

In [91]:
%load_ext cython

Чтобы скопилировать код через Cython внутри notebook есть специальная командая %%cython

In [92]:
%%cython
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return (len(self), len(self[0]))
    
def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # better cell extraction
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

In [93]:
shape = (100, 100)

X = Matrix.random(shape)
Y = Matrix.random(shape)

In [94]:
%timeit matrix_product(X, Y)

34.4 ms ± 35.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Посмотрим, во что именно компилируется код:

In [95]:
%%cython -a
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return (len(self), len(self[0]))

def matrix_product(X, Y):
    n_xrows, n_xcols = X.shape
    n_yrows, n_ycols = Y.shape
    Z = Matrix.zeros((n_xrows, n_ycols))
    Yt = Y.transpose()  # better cell extraction
    for i, (Xi, Zi) in enumerate(zip(X, Z)):
        for k, Ytk in enumerate(Yt):
            Zi[k] = sum(Xi[j] * Ytk[j] for j in range(n_xcols))
    return Z

Чем более желтый цвет, тем менее конкретен тип выражения, а значит меньше оптимизаций возможно. <br>
Сейчас практически все объекты имеют тип `PyObject`

Попробуем переписать умножение матриц на Cython. <br>
К сожалению достаточно сложно переписать версию с транспонированием (так как она очень питонячая). <br>
Поэтому возьмем чуть менее оптимизированную версию.

In [96]:
%%cython -a
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return (len(self), len(self[0]))

cdef matrix_product(X, Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    cdef int acc = 0
    for i from 0 <= i < n_xrows:
        Xi = X[i]
        Zi = Z[i]
        for k from 0 <= k < n_ycols:
            acc = 0
            for j from 0 <= j < n_xcols:
                acc += Xi[j] * Y[j][k]
            Zi[k] = acc
    return Z

In [97]:
%timeit matrix_product(X, Y)

34.5 ms ± 27.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Код на Cython заметно менее желтый и при этом работает быстрее (хотя мы взяли не самую оптимизированную версию). <br>
В целом писать или читать код на Cython довольно легко и он довольно легко позволяет ускорить многие вещи. <br>
Кроме того большее преимущество Cython - то что, это часть стандартного Python, а значит нет никаких проблем с совместимостью.

**Важно:** Для ускорения стоит писать, как на низкоуровневом языке:

In [98]:
%%cython -a
import random

class Matrix(list):
    @classmethod
    def zeros(cls, shape):
        return cls([[0] * shape[1] for i in range(shape[0])])

    @classmethod
    def random(cls, shape):
        M, (n_rows, n_cols) = cls(), shape
        for i in range(n_rows):
            M.append([random.randint(-255, 255)
                      for j in range(n_cols)])
        return M
    
    def transpose(self):
        n_rows, n_cols = self.shape
        return self.__class__(zip(*self))

    @property
    def shape(self):
        return (len(self), len(self[0]))

cdef matrix_product(X, Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    Z = Matrix.zeros((n_xrows, n_ycols))
    
    cdef int acc = 0
    for i from 0 <= i < n_xrows:
        Xi = X[i]
        Zi = Z[i]
        for k from 0 <= k < n_ycols:
            Zi[k] = sum(Xi[j] * Y[j][k] for j in range(n_xcols))
    return Z

In [99]:
%timeit matrix_product(X, Y)

34.7 ms ± 229 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Попробуем ускорить numpy версию

In [100]:
%%cython -a
import numpy as np
cimport numpy as np

def matrix_product(np.ndarray X, np.ndarray Y):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    cdef int acc
    for i from 0 <= i < n_xrows:
        for k from 0 <= k < n_ycols:
            acc = 0
            for j from 0 <= j < n_xcols:
                acc += X[i, j] * Y[j, k]
            Z[i, k] = acc
    return Z

In [101]:
%timeit matrix_product(X3, Y3)

220 ms ± 5.42 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Тип ndarray - не конкретен. Поэтому тело цикла ярко желтое.

In [102]:
%%cython -a
import numpy as np
cimport numpy as np

def matrix_product(
    np.ndarray[np.int64_t, ndim=2] X,
    np.ndarray[np.int64_t, ndim=2] Y
):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    cdef int acc
    for i from 0 <= i < n_xrows:
        for k from 0 <= k < n_ycols:
            acc = 0
            for j from 0 <= j < n_xcols:
                acc += X[i, j] * Y[j, k]
            Z[i, k] = acc
    return Z

In [103]:
%timeit matrix_product(X3, Y3)

497 µs ± 193 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Хочется сделать строчки еще менее желтыми. <br>
Сейчас там проверка за выход цикла и переполнения. <br>
Уберем их.

In [104]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False)
@cython.overflowcheck(False)
def matrix_product(
    np.ndarray[np.int64_t, ndim=2] X,
    np.ndarray[np.int64_t, ndim=2] Y
):
    cdef int n_xrows = X.shape[0]
    cdef int n_xcols = X.shape[1]
    cdef int n_yrows = Y.shape[0]
    cdef int n_ycols = Y.shape[1]
    cdef np.ndarray[np.int64_t, ndim=2] Z
    Z = np.zeros((n_xrows, n_ycols), dtype=X.dtype)
    cdef int acc
    for i from 0 <= i < n_xrows:
        for k from 0 <= k < n_ycols:
            acc = 0
            for j from 0 <= j < n_xcols:
                acc += X[i, j] * Y[j, k]
            Z[i, k] = acc
    return Z

In [105]:
%timeit matrix_product(X3, Y3)

375 µs ± 287 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Часть 5. Пробуем PyPy

<img src="https://upload.wikimedia.org/wikipedia/commons/b/b0/Pypy_logo.png?uselang=ru" />

PyPy - альтернативный интерпретатор Python. <br>
Делает JIT-компиляцию и во многих случаях заметно быстрее CPython. <br>
Однако есть проблемы с совместимостью с внешними либами и свежими версиями Python.

PyPy demo

### Часть 6. Прочее

Проекты, заслуживающего внимания:
1. <a href="https://github.com/yglukhov/nimpy">nimpy</a> - Используем функции на языке Nim из Python
2. <a href="https://pythran.readthedocs.io/en/latest/">Pythran</a> - Другой подход к компиляции кода
3. <a href="https://github.com/pyston/pyston">Pyston</a> - еще один интерпретатор с JIT-компилятором

# Оптимизируем память

## Замеряем память

Замерять память в Python - довольно сложно.

In [106]:
import sys

print(sys.getsizeof([i for i in range(1000000)]))
print(sys.getsizeof([i for i in range(100000)]))

8448728
800984


Кажется, что все работает как надо. Однако:

In [107]:
class SomeClass:
    def __init__(self, i):
        self.i = i
        self.j = i * 2
        
sys.getsizeof([SomeClass(i) for i in range(1000000)])

8448728

Почему-то список из `SomeClass` занимает столько же места как и список целых чисел. <br>
По факту `sys.getsizeof` хорошо работает только для простых типов и встроенных структур.

Q: Что делать? 
A: Использовать профилировщик памяти!

In [108]:
%load_ext memory_profiler
%memit

peak memory: 558.19 MiB, increment: 0.06 MiB


Этот подход тоже не идеален. Он замеряет лишь потребление памяти в один конкретный момент времени. <br>
Поэтому он не может все учитывать, а его результаты будут заметно плавать.

In [109]:
%memit [n for n in range(10000000)]

peak memory: 774.19 MiB, increment: 216.16 MiB


In [110]:
%memit [n for n in range(1000000)]

peak memory: 511.50 MiB, increment: 7.42 MiB


## Можно ли получить memory-leakage в Python

Зависит от того, что считать memory-leakage. Как в C++ - только если явно работать со счетчиком ссылок, так как есть Garbage collection.

Немного подробнее: https://rushter.com/blog/python-garbage-collector/

Однако, можно получить долго-живущие "бесполезные" объекты.

Плюс есть особенности старых версий Python (2.7, до 3.4)

In [127]:
def mutable_argument(arr=[]):
    arr.append(42)
    return a

In [131]:
def unused_variable_in_long_process(arg1, arg2, arg3, unused_variable):
    pass

In [134]:
class ClassCaching:
    cache = {}

    def calc(arg):
        result = cache.get(arg)
        if result is not None:
            return result
        result = do_calc(arg)
        cache[arg] = result
        return result

## Array

`array` позволяет более компактно хранить объекты примитвных типов.

In [111]:
import array

%memit array.array('q', range(10000000))

peak memory: 551.73 MiB, increment: 40.23 MiB


Подробнее про типы: https://docs.python.org/3/library/array.html

## np.array

`np.array` так же хранит объекты определенных типов и занимает меньше места, чем стандартный `list`.

In [112]:
%memit np.arange(10000000)

peak memory: 596.12 MiB, increment: 76.30 MiB


## tuple vs list

In [113]:
sys.getsizeof([i for i in one_million_elements])

8448728

In [114]:
sys.getsizeof(tuple(one_million_elements))

8000040

In [115]:
sys.getsizeof(list(one_million_elements))

8000056

## Slots

Использование `__slots__` позволяет заметно сократить объем занимаемой памяти:

In [116]:
class SomeClass:
    def __init__(self, i):
        self.a = i
        self.b = 2 * i
        self.c = 3 * i
        self.d = 4 * i
        self.e = 5 * i

In [117]:
%memit [SomeClass(i) for i in range(1000000)]

peak memory: 840.52 MiB, increment: 237.48 MiB


In [118]:
class SomeClassSlots:
    __slots__ = ('a', 'b', 'c', 'd', 'e',)
    def __init__(self, i):
        self.a = i
        self.b = 2 * i
        self.c = 3 * i
        self.d = 4 * i
        self.e = 5 * i
                
%memit [SomeClassSlots(i) for i in range(1000000)]

peak memory: 797.20 MiB, increment: 187.28 MiB


Кроме того у `__slots__` есть дополнительный плюс - ускорение времени обращения к аттрибуту

In [119]:
d1 = SomeClass(0)
d2 = SomeClassSlots(0)

def attr_work(obj):
    count = 0
    for i in range(10000):
        count += obj.a + obj.b + obj.c + obj.d + obj.e

In [120]:
%timeit attr_work(d1)

428 µs ± 716 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [121]:
%timeit attr_work(d2)

427 µs ± 181 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Однако со `__slots__` не очень удобно работать при наследовании - необходимо его указывать в каждом классе иерархии.

## bitarray

bitarray - пакет для эффективного хранения набора булевских значений. <br>
Подробнее: https://github.com/ilanschnell/bitarray

In [122]:
import bitarray.util as bu

%memit bu.zeros(10000000)

peak memory: 610.42 MiB, increment: 0.02 MiB


In [123]:
%memit [False for i in range(10000000)]

peak memory: 632.12 MiB, increment: 21.70 MiB


Однако нужно понимать, что на обращение к элементу тратится время.

## range - вычисление вместо хранения

In [124]:
a = range(1, 100000, 3)
print(a[10])
print(len(a))

31
33333


Такой же подход можно использовать и для более сложных последовательностей. <br>
Можно использовать смешанный подход с кешированием результата вычислений.

## Другой полезный инструментарий

1. https://github.com/mgedmin/objgraph
2. https://github.com/zhuyifei1999/guppy3